# Experiment: Working with Dolly
## Last Updated $DATE -  $AUTHOR.

```
Summary of High Level Research Question
```

Try to scope your experiments such you can answer your research question in 1-3 hours.
This is an ideal time block to enter flow / deep work, but short enough that you will still feel 
motivated by a relatively tight feedback loop.

If a problem seems like it needs more time that that, 

### High Level Experiment Design

## Goals:
```
List of specific goals that this experiment seeks to achieve.

This should fall under a few categories:
- Development of Intuition about a _specific_ topic
- Novel Research or Insight that could lead to a publishable result
- Meaningfully explore a topic which could lead to an improvement in product

Guiding principles should understanding, insight, and value creation.
```

## Tasks & Experiment Design

```
A list of specific tasks that are going to be tested 

```


## Outcomes

```
Document high level research findings and how
```


In [1]:
# Install things into ENV
# TODO: Setup up a container and push to docker that contains all these
%pip install git+https://github.com/neelnanda-io/TransformerLens.git
%pip install circuitsvis
%pip install plotly


  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-7ggb70o7
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-7ggb70o7
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 0ffcc8ad647d9e991f4c2596557a9d7475617773
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Generic Set of Imports for MI Research
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
from pprint import pprint
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [3]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from pprint import pprint

In [4]:
# Setup PyTorch configuration for inference based experiments
# NOTE: Mark as False if you want to do any kind of training 
#       as part of your experimentation

INFERENCE_ONLY_EXPERIMENT = True
if INFERENCE_ONLY_EXPERIMENT:
    torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

import plotly.io as pio
pio.renderers.default = "notebook_connected"

cuda


In [5]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [6]:
# Load Circuit Visualizations
# TODO: Explore building out our own packages / tooling
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Vivek")


In [7]:
# Load & Run a Model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-7b")
hf_model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-7b")

print("Loaded hf_model, hooking transformer into TransformerLens!")
# model = HookedTransformer.from_pretrained(
#     "EleutherAI/pythia-6.9b-deduped",
#     center_unembed=False,
#     center_writing_weights=False,
#     fold_ln=False,
#     refactor_factored_attn_matrices=True,
#     hf_model=hf_model
# )

### Janky Shit
### TODO: Figure out how this library actually works and make this a cleaner integration.
import transformer_lens.loading_from_pretrained as loading
# Get the model name used in HuggingFace, rather than the alias.
official_model_name = loading.get_official_model_name("EleutherAI/pythia-6.9b-deduped")


# Load the config into an HookedTransformerConfig object. If loading from a
# checkpoint, the config object will contain the information about the
# checkpoint
cfg = loading.get_pretrained_model_config(
    official_model_name,
    checkpoint_index=None,
    checkpoint_value=None,
    fold_ln=False,
    device=device,
    n_devices=1,
)
print(cfg)
cfg.d_vocab = 50280
cfg.d_vocab_out = 50280
print(cfg)


# Get the state dict of the model (ie a mapping of parameter names to tensors), processed to match the HookedTransformer parameter names.
state_dict = loading.get_pretrained_state_dict(
    official_model_name, cfg, hf_model
)

# Create the HookedTransformer object
model = HookedTransformer(cfg, tokenizer=tokenizer)

model.load_and_process_state_dict(
    state_dict,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    refactor_factored_attn_matrices=False,
    move_state_dict_to_device=True,
)

print(f"Loaded pretrained model into HookedTransformer!")

model_description_text = """For this demo notebook we'll look at Dolly v2. It is based on pythia 6.9b, but we use the weights for dolly v2. To try the model the model out, let's find the loss on this paragraph!"""
# return_type of model can be loss, logits, both, or none!
loss = model(model_description_text, return_type="loss")
print("Model loss:", loss)


Loaded hf_model, hooking transformer into TransformerLens!
HookedTransformerConfig:
{'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 128,
 'd_mlp': 16384,
 'd_model': 4096,
 'd_vocab': 50432,
 'd_vocab_out': 50432,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.0125,
 'model_name': 'pythia-6.9b-deduped',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 32,
 'n_layers': 32,
 'n_params': 6442450944,
 'normalization_type': 'LN',
 'original_architecture': 'GPTNeoXForCausalLM',
 'parallel_attn_mlp': True,
 'positional_embedding_type': 'rotary',
 'rotary_dim': 32,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'EleutherAI/pythia-6.9b-deduped',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_to

In [8]:
# DOLLY V2 - 7B Config
pprint(model.cfg)

# Transformer Lens Note:
# get_token_position, to_tokens, to_string, to_str_tokens, prepend_bos, to_single_token
# are all methods that are added to the model object by TransformerLens

HookedTransformerConfig:
{'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 128,
 'd_mlp': 16384,
 'd_model': 4096,
 'd_vocab': 50280,
 'd_vocab_out': 50280,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.0125,
 'model_name': 'pythia-6.9b-deduped',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 32,
 'n_layers': 32,
 'n_params': 6442450944,
 'normalization_type': 'LN',
 'original_architecture': 'GPTNeoXForCausalLM',
 'parallel_attn_mlp': True,
 'positional_embedding_type': 'rotary',
 'rotary_dim': 32,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'EleutherAI/pythia-6.9b-deduped',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_tokens': False,
 'use_local_attn': False,
 'use_split_qkv_inp

In [9]:
#from easy_transformer.utils import get_corner, gelu_new, tokenize_and_concatenate


In [10]:
print("done")

done


In [11]:
# Direct Logit Attribution


In [12]:
from transformer_lens.utils import get_corner, gelu_new, tokenize_and_concatenate

In [13]:
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train")
from pprint import pprint
batch_size = 8
num_epochs = 1
max_steps = 1
log_every = 1
lr = 1e-3
weight_decay = 1e-2
overfitMax=1

#import gc

#gc.collect()

#torch.cuda.empty_cache()
tokens_dataset = tokenize_and_concatenate(dataset, model.tokenizer, streaming=False, max_length=cfg.n_ctx, column_name="text", add_bos_token=True, num_proc=4)
data_loader = torch.utils.data.DataLoader(tokens_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
optimizer_copy = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

Found cached dataset parquet (/root/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f7df9b7d85d80cc4_*_of_00004.arrow


In [14]:

#cfg.debug = False
print("starting")
torch.set_grad_enabled(True)
test_string_trained = """German police and security services"""
c=1
#print(model.tokenizer.batch_decode(logits).shape)
tokens = model.to_tokens(test_string_trained, prepend_bos=True).cuda()

logits, loss = model(tokens, return_type="both")
#print(logits.shape)
#print(tokens.shape)
#lossF = torch.nn.CrossEntropyLoss()
#loss = lossF(logits, tokens)

#print(list(enumerate(list(model.tokenizer.batch_decode(logits)))))
model.zero_grad(set_to_none=True)
loss.backward()
#optimizer_copy.step()
#optimizer_copy.zero_grad()
#losses.append(loss.item())
#print(f"Step: {c}, Loss: {loss.item():.4f}")

#logits, loss = model(tokens, return_type="both")
#print(model.tokenizer.batch_decode(logits))
#losses.append(loss.item())
print(f"Step: {c}, Loss: {loss.item():.4f}")

          

starting
Step: 1, Loss: 4.1990


In [24]:
#print(model.tokens_to_residual_directions(tokens))
#print(model.tokenizer.batch_decode(
print(model.to_tokens(logits))


TypeError: can only concatenate str (not "Tensor") to str